In [22]:
import requests
import pprint

In [23]:
pp = pprint.PrettyPrinter(indent=2)

In [24]:
song_list = None
with open("data/songs.txt") as f:
    song_list = [x.rstrip() for x in f.readlines()]
song_list[:10]

["Boy's a Liar",
 "That's what I want",
 'Toosie Slide',
 'Savage',
 'abdefu',
 'Thriller',
 'Say So',
 'Mi Gente',
 "U can't touch this",
 'Stay']

In [25]:
def return_song_dump(song_name):
    data = {'name': song_name}
    response = requests.post('http://localhost:3000/search', json=data)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print('Request failed with status code:', response.status_code)
        return -1

In [33]:
def extract_info(dump):
    song_info = {}
    # pp.pprint(dump)
    song_info["artists"] = ", ".join([x['name'] for x in dump['artists']])
    song_info["song_id"] = dump['id']
    song_info["popularity"] = dump['popularity']
    song_info["spotify_link"] = dump['external_urls']['spotify']
    song_info["release_date"] = dump['album']['release_date']
    
    return song_info

In [34]:
song_data = {}

for idx, song_name in enumerate(song_list):
    dump = return_song_dump(song_name)
    song_data[idx] = extract_info(dump[0])
    song_data[idx]['name'] = song_name

In [35]:
def extract_song_features(song_id):
    params = {'id': song_id}
    response = requests.get('http://localhost:3000/features', params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print('Request failed with status code:', response.status_code)
        return -1

In [36]:
song_data[0]

{'artists': 'PinkPantheress, Ice Spice',
 'song_id': '6AQbmUe0Qwf5PZnt4HmTXv',
 'popularity': 98,
 'spotify_link': 'https://open.spotify.com/track/6AQbmUe0Qwf5PZnt4HmTXv',
 'release_date': '2023-02-03',
 'name': "Boy's a Liar"}

In [40]:
for idx, song in song_data.items():
    features = extract_song_features(song['song_id'])
    try:
        for k, v in features.items():
            if k not in song:
                song[k] = v
    except AttributeError:
        print(song)

Request failed with status code: 500
{'artists': 'Naturelle', 'song_id': '5N8KsYJ8IEMQVw33JvytW8', 'popularity': 85, 'spotify_link': 'https://open.spotify.com/track/5N8KsYJ8IEMQVw33JvytW8', 'release_date': '2022-02-08', 'name': 'Falling'}


In [41]:
song_data[0]

{'artists': 'PinkPantheress, Ice Spice',
 'song_id': '6AQbmUe0Qwf5PZnt4HmTXv',
 'popularity': 98,
 'spotify_link': 'https://open.spotify.com/track/6AQbmUe0Qwf5PZnt4HmTXv',
 'release_date': '2023-02-03',
 'name': "Boy's a Liar",
 'danceability': 0.696,
 'energy': 0.809,
 'key': 5,
 'loudness': -8.254,
 'mode': 1,
 'speechiness': 0.05,
 'acousticness': 0.252,
 'instrumentalness': 0.000128,
 'liveness': 0.248,
 'valence': 0.857,
 'tempo': 132.962,
 'type': 'audio_features',
 'id': '6AQbmUe0Qwf5PZnt4HmTXv',
 'uri': 'spotify:track:6AQbmUe0Qwf5PZnt4HmTXv',
 'track_href': 'https://api.spotify.com/v1/tracks/6AQbmUe0Qwf5PZnt4HmTXv',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6AQbmUe0Qwf5PZnt4HmTXv',
 'duration_ms': 131013,
 'time_signature': 4}

In [42]:
import json

In [43]:
with open('data/dance_data.json', 'w') as f:
    json.dump(song_data, f, indent=2)

In [51]:
with open('data/dance_data.json') as f:
    songs_data = json.load(f)

In [52]:
key_scale = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]

In [53]:
def process_song_data(data):
    data['mode'] = "Major" if data['mode'] == 1 else "Minor"
    data['key'] = key_scale[int(data['key'])]
    data['duration'] = data['duration_ms']/1000/60
    data.pop('duration_ms')

In [54]:
for key, data in songs_data.items():
    try:
        process_song_data(data)
    except KeyError:
        print(data)

{'artists': 'Naturelle', 'song_id': '5N8KsYJ8IEMQVw33JvytW8', 'popularity': 85, 'spotify_link': 'https://open.spotify.com/track/5N8KsYJ8IEMQVw33JvytW8', 'release_date': '2022-02-08', 'name': 'Falling'}


In [55]:
with open('data/dance_data_processed.json', 'w') as f:
    json.dump(songs_data, f, indent=2)